In [1]:
# infer_gpt2_chinese_lora.py
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

In [2]:
# 模型路径
base_model = "uer/gpt2-chinese-cluecorpussmall"
lora_model_path = "outputs/gpt2_chinese_lora_small"

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained("outputs/gpt2_chinese_lora_small", use_fast=False) #use_fast和中文兼容不好

# 加载基础模型 + LoRA adapter
model = AutoModelForCausalLM.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, lora_model_path)
model.eval() #进入推理模式，关闭 dropout 和梯度计算

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(21128, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

In [8]:
# 生成函数
def generate(text, max_tokens=50):
    inputs = tokenizer(text, return_tensors="pt").to(model.device) #用 tokenizer 将输入的原始文本转成 token（tensor 格式），例如 [[101, 2345, 102]]；return_tensors="pt" 表示返回 PyTorch 格式；.to(model.device) 把输入送到和模型一样的设备（GPU 或 CPU），防止“设备不一致”的错误。
    with torch.no_grad(): #关闭梯度计算，加速推理；
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True, #使用随机采样而非贪婪解码；
            top_p=0.99, #核采样（只保留前 95% 概率的 token）；
            temperature=0.95 #温度系数（值越小越保守，越大越发散）；
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True) #将输出的 token id 转换成可读文本；skip_special_tokens=True：跳过 <pad>, <bos> 等特殊符号。

In [9]:
# 示例调用
if __name__ == "__main__":
    prompt = "她站在桥头，望着雨雾中的城墙，心里忽然泛起一种奇怪的情绪。"
    output = generate(prompt)
    print("📜 生成内容：\n", output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


📜 生成内容：
 她 站 在 桥 头 ， 望 着 雨 雾 中 的 城 墙 ， 心 里 忽 然 泛 起 一 种 奇 怪 的 情 绪 。 上 官 婉 儿 是 如 何 回 转 着 走 来 的 ？ 是 谁 已 经 走 了 多 少 步 ？ 他 在 桥 上 又 是 走 了 多 少 步 ？ 那 些 人 都 站 在 哪 里 ？ 他 不 再 想 走
